LSTM

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import joblib
from keras.models import load_model

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Preprocesamiento de datos
X = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values
y = data['Reporte de fallo'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Crear el modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# Guardar el modelo
model.save('lstm_model2.h5')

Predicciones del modelo

In [11]:
# Cargar tus datos de prueba (asegúrate de que tengan la misma estructura que los datos de entrenamiento)
test_data = pd.read_csv('appliance_usage_data.csv')
X_test = test_data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Cargar el modelo entrenado
model = load_model('lstm_model2.h5')

# Realizar predicciones
yhat = model.predict(X_test)

print(yhat)

# Aplicar un umbral (opcional) para obtener valores binarios (0's y 1's)
yhat_binary = (yhat >= 0.5).astype(int)

15625/15625 [==============================] - 30s 2ms/step
[[0.498172  ]
 [0.4961025 ]
 [0.49400753]
 ...
 [0.49392575]
 [0.4933683 ]
 [0.49557686]]


In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Preprocesamiento de datos
X = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values
y = data['Reporte de fallo'].values

# Normalizar los datos de entrada
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Normalizar los datos de salida
scaler_y = MinMaxScaler(feature_range=(0, 1))
y = scaler_y.fit_transform(y.reshape(-1, 1))

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Crear el modelo LSTM
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=72, validation_data=(X_test, y_test), verbose=2, shuffle=False)

# Guardando el modelo
model.save('lstm_model.h5')

# Guardando los escaladores
joblib.dump(scaler, 'scaler_lstm.pkl')
joblib.dump(scaler_y, 'scaler_y_lstm.pkl')

# Evaluar el modelo
y_pred = model.predict(X_test)

# Invertir la normalización en las predicciones
y_pred_orig_scale = scaler_y.inverse_transform(y_pred)

print(y_pred_orig_scale)

# Mostrar la precisión del modelo
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f'RMSE: {rmse}')


Epoch 1/10
5556/5556 - 23s - loss: 0.5001 - val_loss: 0.4998 - 23s/epoch - 4ms/step
Epoch 2/10
5556/5556 - 20s - loss: 0.5001 - val_loss: 0.4998 - 20s/epoch - 4ms/step
Epoch 3/10
5556/5556 - 18s - loss: 0.5000 - val_loss: 0.4997 - 18s/epoch - 3ms/step
Epoch 4/10
5556/5556 - 18s - loss: 0.5000 - val_loss: 0.4997 - 18s/epoch - 3ms/step
Epoch 5/10
5556/5556 - 17s - loss: 0.4999 - val_loss: 0.4996 - 17s/epoch - 3ms/step
Epoch 6/10
5556/5556 - 17s - loss: 0.4999 - val_loss: 0.4996 - 17s/epoch - 3ms/step
Epoch 7/10
5556/5556 - 17s - loss: 0.4999 - val_loss: 0.4996 - 17s/epoch - 3ms/step
Epoch 8/10
5556/5556 - 16s - loss: 0.4999 - val_loss: 0.4996 - 16s/epoch - 3ms/step
Epoch 9/10
5556/5556 - 16s - loss: 0.4999 - val_loss: 0.4996 - 16s/epoch - 3ms/step
Epoch 10/10
5556/5556 - 16s - loss: 0.4999 - val_loss: 0.4996 - 16s/epoch - 3ms/step


c:\Users\sebas\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3125/3125 [==============================] - 6s 2ms/step
[[0.04937335]
 [0.04891926]
 [0.04676594]
 ...
 [0.04745354]
 [0.04797097]
 [0.04860092]]
RMSE: 0.6734465895755429


Haciendo predicciones

In [26]:
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import joblib

# Cargando los escaladores.
scaler = joblib.load('scaler_lstm.pkl')
scaler_y = joblib.load('scaler_y_lstm.pkl')

# Carga tus datos desde un archivo CSV
data = pd.read_csv('appliance_usage_data1.csv')

# Selecciona las columnas que quieres usar como datos de entrada
input_data = data[['Hora', 'Watts usados', 'Porcentaje de fallos']].values

input_data = scaler.transform(input_data)

# Cambiar la forma de los datos para LSTM (muestras, pasos de tiempo, características)
input_data = input_data.reshape((input_data.shape[0], 1, input_data.shape[1]))

# Cargar el modelo
model = load_model('lstm_model.h5')

# Hacer una predicción
predictions = model.predict(input_data)

# Invirtiendo la normalización en las predicciones
predictions_orig_scale = scaler_y.inverse_transform(predictions)

# Convertir las predicciones a 0's y 1's
y_pred_binary = np.where(y_pred > 0.05, 1, 0)

print(y_pred_binary)

# Guardar las predicciones en un archivo txt
with open('predictions.txt', 'w') as f:
    for prediction in y_pred_binary:
        f.write(f'{prediction}\n')

15625/15625 [==============================] - 30s 2ms/step
[[1]
 [1]
 [0]
 ...
 [0]
 [1]
 [1]]
